# Importing Libraries

In [1]:
from numpy.lib.shape_base import split
import pandas as pd
import numpy as np

# 1/Decision Tree from Scratch

# Entropy, Information Gain, Maximum Feature Information Gain

In [2]:
def Ent(data, clsf): #Calculating the entropy of each class and sum up
    entropy = 0
    #num of all data
    num_all = data.shape[0] 
    ent=0;
    for t in clsf:
        clsfNum = data[data[Targetfeat] == t].shape[0] 
        clsfEnt = 0
        if clsfNum != 0:
            clsfProb = clsfNum / num_all
            clsfEnt = - clsfProb * np.log2(clsfProb)
        ent += clsfEnt
    return ent


def Information_Gain(featName, data, clsf): #Calculating I.G by subtracting total entropy from specific feature information    
    feat_vals = np.unique(data[featName]) 
    num_all = data.shape[0] 
    feat_information = 0.0
    
    for val in feat_vals:
        valData = data[data[featName] == val]
        valNum = valData.shape[0]
        valEntropy = Ent(valData, clsf) 
        valProbabilty = valNum / num_all
        feat_information += valProbabilty * valEntropy 
        ent = Ent(data, clsf)
    return ent - feat_information

def Max_Info_Feature(data, clsf_list):
    features = data.columns.drop(Targetfeat)
    max_info_gain = -1
    max_info_feature = None
    for f in features:
        featureIG = Information_Gain(f, data, clsf_list)
        if max_info_gain < featureIG:
            max_info_gain = featureIG
            max_info_feature = f
    return max_info_feature


# Bulding Desicion Tree and Designing Recursive ID3 Algorithm

In [3]:
def Raw_Tree(featName, data, clsf, depth):
    
    feat_vals_nums = data[featName].value_counts(sort=False)
    # Initilizing Tree
    tree = {} 
    
    for val, num in feat_vals_nums.iteritems():
        valData = data[data[featName] == val]
        flaged = False #flag for tracking value of feature is pure class or not
        max_clsf_name = ''
        max_clsf_num = 0
        
        for c in clsf:
            clsf_count = valData[valData[Targetfeat] == c].shape[0]
            
            if clsf_count == num: 
                tree[val] = c 
                data = data[data[featName] != val] #removing rows with feature_value
                flaged = True
            if clsf_count > max_clsf_num:
                max_clsf_num = clsf_count
                max_clsf_name = c

        if (flaged==0):
            if depth != DEPTH-1:
                tree[val] = "?"
            else:
                tree[val] = max_clsf_name         
    return tree, data


def Recurv_ID3(root, prev_feat_val, data, clsf, depth): #ID3 recursive algorithm
    
    if depth == DEPTH:
        return

    if data.shape[0] != 0: 
        max_inf_feat = Max_Info_Feature(data, clsf)
        tree, data = Raw_Tree(max_inf_feat, data, clsf, depth)
        next_root = None
        
        if prev_feat_val != None: 
            root[prev_feat_val] = dict()
            root[prev_feat_val][max_inf_feat] = tree
            next_root = root[prev_feat_val][max_inf_feat]
        else: 
            root[max_inf_feat] = tree
            next_root = root[max_inf_feat]
        
        for node, branch in list(next_root.items()): 
            if branch == "?":
                feat_val_data = data[data[max_inf_feat] == node] 
                Recurv_ID3(next_root, node, feat_val_data, clsf, depth+1)


def Generate_Tree(data):
    tree = {}
    clsf = data[Targetfeat].unique()
    Recurv_ID3(tree, None, data, clsf, 0)
    return tree



# Spliting Data

In [4]:
def Database_Spliting(data): # spliting data in 80-20 order for train and test 
    num_all = data.shape[0] 
    split_ind = int(80 / 100 * num_all)
    test_data = data.iloc[split_ind:].reset_index(drop=True)
    train_data = data.iloc[0:split_ind].reset_index(drop=True)
    return test_data, train_data

# Prediction, Confusion Matrix & Accuracy

In [5]:
def Predict(features, tree): 
        for key in list(features.keys()):
            if key in list(tree.keys()):
                result = tree[key][features[key]]
                if isinstance(result, dict):
                    return Predict(features, result)
                else:
                    return result

def Confusion_Matrix(prediction, targets): 
    conf = np.zeros((num_types, num_types), dtype=np.int32)
    Prediction = np.array(prediction)
    Target = np.array(targets)
    
    for i in range(len(Prediction)):
        if Prediction[i] == 1: 
            if Target[i] == 1:
                conf[0][0] += 1
            elif Target[i] == 0:
                conf[0][1] += 1
        elif Prediction[i] == 0:
            if Target[i] == 1:
                conf[1][0] += 1
            elif Target[i] == 0:
                conf[1][1] += 1
    print(conf)

def Test_ConfusionMatrix(test, tree):
    features_test = test.iloc[:,:-1] #Droping Target Column 
    #print (features_test)
    features_test = features_test.to_dict(orient = "records")
    #print (features_test)
    prediction = pd.DataFrame(columns=["Predict"]) 
    for i in range(len(test)):
        prediction.loc[i,"Predict"] = Predict(features_test[i], tree)
    print ('Accuracy: ',(np.sum(prediction["Predict"] == test[Targetfeat])/len(test))*100)
    Confusion_Matrix(prediction, test[Targetfeat])
    return prediction["Predict"]

# Loading Data and Preparing 

In [6]:
data = pd.read_csv(r'E:\university\Term_7\AI\HW\HW2\HW2\prison_dataset.csv') #load dataset

featNum = data.shape[1] #number of columns(features)
featNames = list(data.columns) #names of columns(features)

data = data.sample(frac = 1).reset_index(drop = True)

# Defining Global Variables (Depth and Target Feature) & Terminal 

In [13]:
DEPTH = 3; 
Targetfeat = 'Recidivism - Return to Prison numeric'
num_types = data[Targetfeat].nunique()

Test_Data, Training_Data = Database_Spliting(data)
Decision_Tree = Generate_Tree(Training_Data.copy())
print (Decision_Tree)
ACC = Test_ConfusionMatrix(Test_Data, Decision_Tree)

{'Fiscal Year Released': {2010: 1, 2013: {'Age At Release': {'>45': {'Convicting Offense Type': {'Other': 1, 'Violent': 1, 'Drug': 1}}, '<45': {'Main Supervising District': {'3JD': 1, '5JD': 1}}}}, 2015: {'Release Type': {'Parole': {'Age At Release': {'>45': 0, '<45': 0}}, 'Discharged End of Sentence': {'Main Supervising District': {'3JD': 0, '5JD': 0}}}}}}
Accuracy:  72.86871961102108
[[1088  198]
 [ 639 1160]]


# 2/ Random Forest

# Spliting Train Data to 4 Parts 

In [10]:
def RandomF_Prep(data): # spliting data to 4 parts 
    num_all = data.shape[0] 
    split_ind = int(25 / 100 * num_all)
    train_data1 = data.iloc[0:split_ind].reset_index(drop=True)
    train_data2 = data.iloc[split_ind:2*split_ind].reset_index(drop=True)
    train_data3 = data.iloc[2*split_ind:3*split_ind].reset_index(drop=True)
    train_data4 = data.iloc[3*split_ind:4*split_ind].reset_index(drop=True)
    return train_data1, train_data2, train_data3, train_data4

# Designing Random Forest & Calculating Accuracy and Confusion Matrix

In [11]:
def Random_Forest(P1,P2,P3,P4):
    majority = pd.DataFrame(columns=["Predict"]) 
    P5 = P1+ P2+ P3+ P4
    for i in range(len(P1)):
        if (P5[i]>2):
            majority.loc[i,"Predict"]=1
        else:
            majority.loc[i,"Predict"]=0
    return majority

def RandomForest_Test_ConfusionMatrix(prediction,test):
    print ('Accuracy: ',(np.sum(prediction["Predict"] == test[Targetfeat])/len(test))*100)
    Confusion_Matrix(prediction, test[Targetfeat])    

# Applying the "From Scratch" Random Forest Algorithm 

In [12]:
data_F = pd.read_csv(r'E:\university\Term_7\AI\HW\HW2\HW2\prison_dataset.csv') #load dataset
data_F = data_F.sample(frac = 1).reset_index(drop = True)
Test_Data_F, Training_Data_F = Database_Spliting(data_F)
train_data1, train_data2, train_data3, train_data4 = RandomF_Prep(Training_Data_F)

Tree1 = Generate_Tree(train_data1.copy())
P1 = Test_ConfusionMatrix(Test_Data_F, Tree1)
Tree2 = Generate_Tree(train_data2.copy())
P2 = Test_ConfusionMatrix(Test_Data_F, Tree2)
Tree3 = Generate_Tree(train_data3.copy())
P3 = Test_ConfusionMatrix(Test_Data_F, Tree3)
Tree4 = Generate_Tree(train_data4.copy())
P4 = Test_ConfusionMatrix(Test_Data_F, Tree4)

Random_F = Random_Forest (P1,P2,P3,P4)
RandomForest_Test_ConfusionMatrix(Random_F,Test_Data_F)


Accuracy:  72.99837925445705
[[1337  435]
 [ 398  915]]
Accuracy:  72.15559157212319
[[1297  421]
 [ 438  929]]
Accuracy:  70.98865478119936
[[1225  385]
 [ 510  965]]
Accuracy:  73.22528363047002
[[1236  327]
 [ 499 1023]]
Accuracy:  73.54943273905997
[[1260  341]
 [ 475 1009]]


# Applying the "Library Base" Random Forest Algorithm 

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

dataset = pd.read_csv(r'E:\university\Term_7\AI\HW\HW2\HW2\prison_dataset.csv')
Test_Data, Training_Data = Database_Spliting(data)

train_features = Training_Data.iloc[:,:-1]
test_features = Test_Data.iloc[:,:-1]
train_targets = Training_Data.iloc[:,-1]
test_targets = Test_Data.iloc[:,-1]

for f in train_features.keys():
    train_features[f] = le.fit_transform(train_features[f])
    test_features[f] = le.fit_transform(test_features[f])

tree = DecisionTreeClassifier(criterion = 'entropy',max_depth = 3).fit(train_features,train_targets)
prediction = tree.predict(test_features)

print("Accuracy: ",tree.score(test_features,test_targets)*100)
Confusion_Matrix(prediction, test_targets)

Accuracy:  70.34035656401944
[[1101  253]
 [ 662 1069]]
